titulos # - subtitulos  ##

# ***<p style="text-align:center;">Análise de Texto através de Bases de Dados Vetoriais</p>***

## Resumo

Este relatório representa o processo de desenvolvimento e implementação de uma Base de Dados Vetorial com o objetivo de serem identificados padrões de texto.

Como Base de Dados vetorial decidimos utilizadr **ChromaDB** devido à sua excelente integração com Python aliada com a sua facilidade de configuração.  
O Projeto dividiu-se essencialmente em 4 fases. **Aquição e Processamento de Dados**, **Geração de Embeddings**, **Armazenamento Vetorial** e **Identificação de Padrões**. Em seguida abordamos cada uma detalhadamente.

Atualmente, é possível carregar variados tipos de ficheiros e pesquisar por um termo ou texto sendo obtido os ficheiros com o conteúdo mais similar.

## Introdução

### importação biblioteca para usar o sistemas de base de dados Chromadb

In [6]:
import chromadb
from chromadb.utils import embedding_functions

importação de funcionalidades cridas fazer a leitura de arquivos a ser usados durante este projeto

In [3]:
import leitor

Importação de bibliotecas necessárias para obter a data e hora com o objetivo fututo de realizar o "benchmarking" dos diversos modelos de conversão de embeddings  

In [ ]:
import os

None


Criar o cliente para o banco de dados ChromaDB

In [7]:
chroma_client = chromadb.Client(
    chromadb.config.Settings(chroma_server_host="chroma", chroma_server_http_port="8000")
)

Escolher o modelo de embeddings, para a conversão do material fornecido pelo Utizador nos respetivos ficheiros "data" em embeddings vetoriais

In [9]:
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

Neste ponto podemos criar a coleção de embeddings numa estrutura de dados predefinida do ChromaDB.

Uma coleção no ChromaDB é um agrupamento com nome de embeddings, dos seus documentos correspondentes e dos respetivos metadados, onde ocorre o processo de organização, gerir e, de forma crucial, efetuar pesquisas por similaridade em dados vetorizados

No caso do ChromaDB, uma coleção é o local fundamental onde se armazenam e organizam os embeddings vetoriais, os dados originais que estes representam, no caso do nosso projeto trabalho com dados do tipo .doc, .pdf e .txt, e os metadados associados.

Como é referido anteriormente, também é possivel trabalhar com outros tipo de dados, tendo o exemplo das imagens,

Assim sendo, na criação da coleção, é necessário definir o nome da coleção  ( name="my_collection" ), o modelo de embeddings ( embedding_function=embedder, sendo o embedder definido pelo Utilizador anteriormente) a utilizar e os metadados que se pretende associar a cada embedding.



In [10]:
collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=embedder, 
        metadata={
        "description": "my first Chroma collection",
        "created": str(datetime.now())
        } 
)

Após a criação da coleção, criamos um dicionário vazio para armazenar os metadados associados a cada embedding, onde o nome do ficheiro é a chave e o conteúdo do ficheiro é o valor.

In [11]:
dados = {}

Função para adicionar os dados lidos dos ficheiros à coleção.

In [12]:
def adiciona_dados(collection, dados):
        # Adicionar ficheiros à coleção
        collection.add(
            documents=list(dados.values()),  # lista de ficheiros
            ids=[k for k in dados.keys()]    # ids dos ficheiros
        )
        print(f"Foram adicionados {len(dados)} ficheiros.")
        # Limpar o dicionário para evitar adicionar os mesmos ficheiros novamente
        dados.clear()

Ler ficheiros pdf

In [ ]:
leitor.extract_text(r"data\pdf", dados)
adiciona_dados(collection, dados)

FileNotFoundError: [WinError 3] O sistema não conseguiu localizar o caminho especificado: 'data\\pdf'

Ler ficheiros docx

In [25]:
leitor.extract_text(r"data\txt", dados)
adiciona_dados(collection, dados)

FileNotFoundError: [WinError 3] O sistema não conseguiu localizar o caminho especificado: 'data\\txt'

Ler ficheiros txt

In [ ]:
leitor.extract_text(r"data\docx", dados)
adiciona_dados(collection, dados)

Com a criação do dicionario realiazada, prpseguimos ao adicionar os metadados à coleção, utilizando o método add da coleção, onde passamos os embeddings, os metadados e o nome da coleção.
A função add() é responsável por adicionar os embeddings à coleção, associando-os aos metadados correspondentes. O parâmetro "ids" é utilizado para identificar cada embedding de forma única, enquanto o parâmetro "documents" contém os dados originais que foram convertidos em embeddings. 

In [ ]:
collection.add(
    documents=list(dados.values()),  # lista de ficheiros
    ids=[k for k in dados.keys()]    # ids dos ficheiros
)

Neste momento, com o objetivo de realizar o "benchmarking" dos diversos modelos de conversão de embeddings, é necessário guardar a data e hora em que os embeddings foram adicionados à coleção. Para isso, utilizamos a biblioteca datetime para obter a data e hora atuais e armazená-las em uma variável chamada "start_rime", que sera inicializada antes do processo de obter os resultados da pesquisa por similaridade.

In [ ]:
start_time = time.perf_counter()

finalmente, o parâmetro "results" é onde realiazamos a pesquisa por similaridade, onde passamos o embedding que queremos pesquisar e o número de resultados que queremos obter. O resultado da pesquisa é armazenado na variável "results", que contém os embeddings mais semelhantes encontrados na coleção.

A pesquisa por similaridade é uma operação fundamental em sistemas de recuperação de informações, onde o objetivo é encontrar os documentos mais relevantes com base em um vetor de consulta. O ChromaDB permite realizar essa pesquisa de forma eficiente, utilizando técnicas avançadas de indexação e recuperação.

Para a sua execução, usamos o método query da coleção, onde passamos o embedding que queremos pesquisar e o número de resultados que queremos obter, ambos à escolha do utilizador. O resultado da pesquisa é armazenado na variável "results", que contém os embeddings mais semelhantes encontrados na coleção.

O Utilizador pode escolher o Termo de pesquisa, desde que se seja uma string, para encontrar os resultados mais semelhantes, ou seja, o termo de pesquisa é a string que o utilizador quer pesquisar na coleção de embeddings.

In [ ]:
prompt = input("Digite o termo que deseja pesquisar: ")

o Utilizador pode escolher o número de resultados que deseja obter, permitindo ajustar a pesquisa de acordo com suas necessidades específicas. Isso é especialmente útil em cenários onde o usuário está interessado em encontrar os documentos mais relevantes ou semelhantes a um determinado embedding de consulta.

In [ ]:
n_resultados = int(input("Quantos resultados deseja obter:  "))

In [ ]:
results = collection.query(
        query_texts=[prompt],    # o que o utilizador quer pesquisar                  
        n_results=n_resultados   # Número de resultados a serem retornados
)

logo apos a pesquisa, é necessário guardar a data e hora em que os embeddings foram adicionados à coleção. Para isso, utilizamos a biblioteca datetime para obter a data e hora atuais e armazená-las em uma variável chamada "end_time" e realizmos o calculo do tempo total de execução, subtraindo o tempo de início ao tempo de término. O resultado é armazenado na variável "query_time", que representa o total da duração para realizar a pesquisa por similaridade.

In [ ]:
end_time = time.perf_counter()
query_time = end_time - start_time
print(f"Modelo de Embedding: all-MiniLM-L6-v2\nTempo de pesquisa: {query_time:.4f} segundos")

In [ ]:
if results and results.get('documents') and len(results['documents'][0]) > 0:
        num_results = len(results['documents'][0])
        doc_text = []
        titles = []
        distance = []
        for i in range(num_results):
            doc_text.append(results['documents'][0][i])
            distance.append(results['distances'][0][i])
            doc_id_path = results['ids'][0][i]
            titles.append(os.path.basename(doc_id_path))

        leave = False
        while not leave:
            print(f"\n--- Pesquisas encontradas ---\n")   
            j = 0 
            for title in titles:
                print(f"--- Resultado {j+1} ---")
                print(f"Ficheiro: {title}")
                print(f"Distância de Similaridade: {distance[j]:.4f}\n")
                j += 1
                
            print("Escolha um resultado para ver o conteúdo ou escreva 0 para sair.")
            choice = input("Digite o número do resultado desejado: ")
            if choice == '0':
                leave = True
            else:
                try:
                    choice = int(choice) - 1  # Ajustar para índice 0
                    if 0 <= choice < num_results:
                        print("============================================\n")
                        print(f"Conteúdo do Resultado {choice + 1}:")
                        print(f"Ficheiro: {titles[choice]}")
                        print(f"{doc_text[choice]}\n\n")
                        
                    else:
                        print("Opção inválida!")
                except ValueError:
                    print("Por favor, escreva apenas números.")
else:
        print("Nenhum resultado encontrado.")



Para finalizar, é necessário imprimir os resultados da pesquisa, onde apresentamos o nome do ficheiro, o conteúdo do ficheiro e o tempo total de execução. Para isso, utilizamos um loop para iterar sobre os resultados e imprimir as informações desejadas. O resultado final é uma lista dos documentos mais semelhantes encontrados na coleção, juntamente com o tempo total de execução da pesquisa.

Para finalizar, é necessario limpar a coleção, para isso utilizamos o método delete da coleção, onde passamos o nome da coleção que queremos eliminar. O resultado é uma coleção vazia, pronta para ser utilizada novamente.

In [ ]:
ids = collection.get(include=[])
for id in ids['ids']:
    print(id)
id = input("Digite o ID do ficheiro que deseja eliminar: ")
if id in ids['ids']:
    collection.delete(ids=[id])
    print(f"Ficheiro {id} eliminado com sucesso.")
else:
    print(f"Ficheiro {id} não encontrado.")

Dentro da pasta app, existe um ficheiro chamado main.py, que é o ponto de entrada da aplicação. Este ficheiro contém o código principal que executa a aplicação e inicia um menu interativo para o utilizador. O menu permite ao utilizador escolher entre diferentes opções, como adicionar novos embeddings à coleção, realizar pesquisas por similaridade ou limpar a coleção existente.